In [1]:
import acquire
import pandas as pd
from string import digits
import explore
import requests
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd

import re
import unicodedata
import nltk

from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
df = acquire.get_github_geology_results(cached=True)
df.head()

,language,content,date
0,Jupyter Notebook,hylite\nhylite is an open-source python packag...,2020-11-27 17:31:07.762
1,Python,\n\n\n\n\n\nWebviz subsurface\n\n✨👓 Live demo ...,2020-11-27 17:31:08.305
2,R,"Reforester\nReforester, an R program that:\n\n...",2020-11-27 17:31:08.926
3,JavaScript,Swissgeol NGM\nA Geology 3D viewer\nSwissgeol ...,2020-11-27 17:31:09.532
4,Python,geomodel-2-3dweb\n\nGenerates 3D web versions ...,2020-11-27 17:31:10.206


In [3]:
###################################### Clean Data Function ###############################

def basic_clean(string):
    '''
    Converts text in to ascii to remove special characters, then converts back in to utf-8
    '''
    string = (unicodedata.normalize('NFKD', string.lower())
            .encode('ascii', 'ignore') # ascii to reduce noise
            .decode('utf-8', 'ignore') # decode using utf-8
           )
    string = re.sub(r"[^a-z0-9\s]", '', string)
     # Remove numbers from text
    string = re.sub(r'\d+', '', string)
    return string
    

def tokenize(string):
    '''
    This function takes in a string and
    returns a tokenized string.
    '''
    # Create tokenizer.
    tokenizer = nltk.tokenize.ToktokTokenizer()
    
    # Use tokenizer
    string = tokenizer.tokenize(string, return_str=True)
    
    return string

def lemmatize(string):
    '''
    This function takes in string for and
    returns a string with words lemmatized.
    '''
    # Create the lemmatizer.
    wnl = nltk.stem.WordNetLemmatizer()
    
    # Use the lemmatizer on each word in the list of words we created by using split.
    lemmas = [wnl.lemmatize(word) for word in string.split()]
    
    # Join our list of words into a string again and assign to a variable.
    string = ' '.join(lemmas)
    
    return string


def remove_stopwords(string):
    '''
    This function takes in a string, optional extra_words and exclude_words parameters
    with default empty lists and returns a string.
    '''
    # Create stopword_list.
    stopword_list = stopwords.words('english')
    #print(f"Lenght of stopword list before:{len(stopword_list)}")
    
    # Split words in string.
    words = string.split()
    
    # Create a list of words from my string with stopwords removed and assign to variable.
    filtered_words = [word for word in words if word not in stopword_list]
    
    # Join words in the list back into strings and assign to a variable.
    string_without_stopwords = ' '.join(filtered_words)
    return string_without_stopwords

    
def clean_data(df):
    '''
    This function takes in a dataframe of text, cleans, tokenizes, lemmatizes, and removes stopwords
    from that text, appending each step in the process to the dataframe.  It also appends a list of 
    words from each article as well as the total lenght.  
    '''    

    # Formatts repo contents to make them easier to read
    df['text_cleaned'] = df.content.apply(basic_clean)
    df['text_tokenized'] = df.text_cleaned.apply(tokenize)
    df['text_lemmatized'] = df.text_tokenized.apply(lemmatize)
    df['text_filtered'] = df.text_lemmatized.apply(remove_stopwords)
    # Add column with list of words
    words = [re.sub(r'([^a-z0-9\s]|\s.\s)', '', doc).split() for doc in df.text_filtered]
    df = pd.concat([df, pd.DataFrame({'words': words})], axis=1)
    # Adds column with lenght of word list
    df['doc_length'] = [len(wordlist) for wordlist in df.words]
    df = df[['language','text_filtered','words','doc_length']]
    
    # removing unpopular languages 
    #language_list = ['JavaScript', 'R', 'Jupyter Notebook']
    #df = df[df.language.isin(language_list)]
    return df

In [4]:
clean_data(df)

,language,text_filtered,words,doc_length
0,Jupyter Notebook,hylite hylite opensource python package prepro...,"[hylite, hylite, opensource, python, package, ...",300
1,Python,webviz subsurface live demo application introd...,"[webviz, subsurface, live, demo, application, ...",261
2,R,reforester reforester r program us logistic bi...,"[reforester, reforesterprogram, us, logistic, ...",771
3,JavaScript,swissgeol ngm geology viewer swissgeol new geo...,"[swissgeol, ngm, geology, viewer, swissgeol, n...",120
4,Python,geomodeldweb generates web version geological ...,"[geomodeldweb, generates, web, version, geolog...",174
...,...,...,...,...
1836,Jupyter Notebook,hylite hylite opensource python package prepro...,"[hylite, hylite, opensource, python, package, ...",300
1837,Python,webviz subsurface live demo application introd...,"[webviz, subsurface, live, demo, application, ...",261
1838,R,reforester reforester r program us logistic bi...,"[reforester, reforesterprogram, us, logistic, ...",771
1839,JavaScript,swissgeol ngm geology viewer swissgeol new geo...,"[swissgeol, ngm, geology, viewer, swissgeol, n...",120
